In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile,isdir, join

mypath = 'C:/Users/Dell/Desktop/EEG Arithmetic/Features'

In [2]:
X =[]
onlydir = [d for d in listdir(mypath) if isdir(join(mypath, d))] 
l=[]
for dir_ in onlydir:
    trial = ['Trial 1','Trial 2']
    for i in trial:
        onlyfiles = [f for f in listdir(mypath+'/'+dir_+'/'+i+'/') if isfile(join(mypath+'/'+dir_+'/'+i+'/', f)) and '.xlsx' in f]
        df = pd.read_excel(mypath+'/'+dir_+'/'+i+'/'+onlyfiles[0])
        df['subject'] = dir_
        if i == 'Trial 1':
            df['trial'] = 0
        if i == 'Trial 2':
            df['trial'] = 1
        l.append(df)
        

df_ = pd.concat(l)
df_['Area_mag'] =  df_['Area'].apply (lambda row: abs(complex(row.replace(" ", ""))))
#df_.drop(df_[df_['SODP_No'] > 5].index, inplace = True)
df_.drop(columns=['Area'], inplace= True)
df_.drop(columns=['Trial'], inplace= True)
df_.reset_index(drop = True, inplace = True)
df_.head()

,Channel,SODP_No,m(r=0.5),ctm(r=0.5),subject,trial,Area_mag
0,EEG Fp1,1,0.179768,0.284309,0,0,28.116831
1,EEG Fp1,2,0.217262,0.254318,0,0,25.110411
2,EEG Fp1,3,0.504542,0.247455,0,0,4.568114
3,EEG Fp1,4,0.890418,0.204030,0,0,2.884154
4,EEG Fp1,5,0.995763,0.122184,0,0,1.455482


In [3]:
y = []
label_path = 'C:/Users/Dell/Desktop/EEG Arithmetic/subject-info.csv'
label_file = pd.read_csv(label_path)
label_file = label_file[['Subject','Gender','Count quality']]

In [4]:
label_file['Subject']= [int(i[7:9]) for i in label_file['Subject']]

In [5]:
label_file['Gender'] = [0 if i =='F' else 1 for i in label_file['Gender']]
label_file.head()

,Subject,Gender,Count quality
0,0,0,0
1,1,0,1
2,2,0,1
3,3,0,1
4,4,0,0


In [6]:
count_quality = []
gender = []
for i in df_.index:
    sub = int(df_.loc[i]['subject'])
    count_quality.append(label_file.loc[label_file['Subject'] == sub]['Count quality'].values[0])
    gender.append(label_file.loc[label_file['Subject'] == sub]['Gender'].values[0])

df_['count_quality'] = count_quality
df_['gender'] = gender
df_.head()

,Channel,SODP_No,m(r=0.5),ctm(r=0.5),subject,trial,Area_mag,count_quality,gender
0,EEG Fp1,1,0.179768,0.284309,0,0,28.116831,0,0
1,EEG Fp1,2,0.217262,0.254318,0,0,25.110411,0,0
2,EEG Fp1,3,0.504542,0.247455,0,0,4.568114,0,0
3,EEG Fp1,4,0.890418,0.204030,0,0,2.884154,0,0
4,EEG Fp1,5,0.995763,0.122184,0,0,1.455482,0,0


In [12]:
##Classifications
l = df_.Channel.unique()
print(len(l))
l

21


array(['EEG Fp1', 'EEG Fp2', 'EEG F3', 'EEG F4', 'EEG F7', 'EEG F8',
       'EEG T3', 'EEG T4', 'EEG C3', 'EEG C4', 'EEG T5', 'EEG T6',
       'EEG P3', 'EEG P4', 'EEG O1', 'EEG O2', 'EEG Fz', 'EEG Cz',
       'EEG Pz', 'EEG A2-A1', 'ECG ECG'], dtype=object)

In [26]:
len(df_.loc[df_['Channel'] != l[20]])

10698

In [289]:
X = df_.loc[df_['Channel'] == l[20]][['m(r=0.5)','ctm(r=0.5)','subject','Area_mag']]
y = df_.loc[df_['Channel'] == l[20]][['trial']].values
y = y.ravel()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [290]:
from sklearn.model_selection import GridSearchCV 
from sklearn import svm

# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.526, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.526, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.526, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.526, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.526, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.526, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0

[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.438, total=   0.0s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.401, total=   0.0s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.467, total=   0.0s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=1000, gamma=0.1, kernel=rbf, score=0.453, total=   0.0s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=1000, gamma=0.1, kernel=rbf, score=0.511, total=   0.0s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=1000, gamma=0.1, kernel=rbf, score=0.431, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=0.445, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.6s finished


In [291]:
from sklearn.model_selection import RandomizedSearchCV 
# Instantiating RandomizedSearchCV object 
tree_cv = RandomizedSearchCV(svm.SVC(), param_grid, cv = 5) 
  
tree_cv.fit(X_train, y_train) 
  
# Print the tuned parameters and score 
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_)) 
print("Best score is {}".format(tree_cv.best_score_)) 
print(tree_cv.best_estimator_) 

Tuned Decision Tree Parameters: {'kernel': 'rbf', 'gamma': 0.001, 'C': 0.1}
Best score is 0.5255474452554745
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [292]:
clf = grid.best_estimator_ #tree_cv.best_estimator_ #

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_train)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_train, y_pred))
print("F1_Score:",metrics.f1_score(y_train, y_pred, labels=np.unique(y_pred),  average = 'micro'))
print("F1_Score:",metrics.f1_score(y_train, y_pred, labels=np.unique(y_pred),  average = 'macro'))
print("F1_Score:",metrics.f1_score(y_train, y_pred, labels=np.unique(y_pred),  average = 'weighted'))
print("Confusion Metrics:",metrics.confusion_matrix(y_train, y_pred))
print( )

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1_Score:",metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred),  average = 'micro'))
print("F1_Score:",metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred),  average = 'macro'))
print("F1_Score:",metrics.f1_score(y_test, y_pred, labels=np.unique(y_pred),  average = 'weighted'))
print("precision_score:",metrics.precision_score(y_test, y_pred))
print("recall_score:",metrics.recall_score(y_test, y_pred))
print("Confusion Metrics:",metrics.confusion_matrix(y_test, y_pred))

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
#print("Mean_Squared_Error:",metrics.mean_squared_error(y_test, y_pred))


Accuracy: 0.5255474452554745
F1_Score: 0.6889952153110048
F1_Score: 0.6889952153110048
F1_Score: 0.6889952153110049
Confusion Metrics: [[216   0]
 [195   0]]

Accuracy: 0.427536231884058
F1_Score: 0.5989847715736041
F1_Score: 0.5989847715736041
F1_Score: 0.598984771573604
precision_score: 0.0
recall_score: 0.0
Confusion Metrics: [[59  0]
 [79  0]]
59 0 79 0


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
